$\newcommand{\To}{\Rightarrow}$

In [1]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [2]:
from kernel.type import HOLType, TVar, Type, TFun, boolT
from data.nat import natT

## Types

In higher-order logic, every term has a type. Common types include booleans, natural numbers, functions, lists, and so on. We also need the concept of *type variables*. Types are implemented in `kernel/type.py`.

Booleans and natural numbers are type constants that do not take any parameters. They can be constructed as follows:

In [3]:
print(Type("bool"))

bool


In [4]:
print(Type("nat"))

nat


We use `boolT` as a shorthand for `Type("bool")`, and `natT` as a shorthand for `Type("nat")`.

In [5]:
print(boolT)
print(natT)

bool
nat


Functions is a very important class of types. Given any two types $A$ and $B$, the type $A \To B$ represents functions from $A$ to $B$. For example, the type $nat \To bool$ represents functions from natural numbers to booleans, or in other words, properties of natural numbers. This type is constructed as follows:

In [6]:
print(Type("fun", natT, boolT))

nat => bool


A shortcut to construct function types is to use `TFun`:

In [7]:
print(TFun(natT, boolT))

nat => bool


A key concept for dealing with function types is *currying*. It allows us to represent functions of multiple arguments. For example, the type of functions taking two natural numbers as arguments, and output one natural number, is given by $nat \To (nat \To nat)$. Note this is very different from $(nat \To nat) \To nat$. Since the former is used more frequently, we have the convention that the operator $\To$ associates to the right, so the former type is simply written as $nat \To nat \To nat$. In general, the type $A_1 \To \cdots \To A_n \To C$ can be read as: functions taking arguments of type $A_1,\dots A_n$ as input, and output a value of type $C$.

In [8]:
print(TFun(natT, TFun(natT, natT)))

nat => nat => nat


`TFun` can actually take any number of arguments:

In [9]:
print(TFun(natT, natT, natT))

nat => nat => nat


Functions are not the only types with arguments. Given any type $A$, we can form the type of (finite) lists with entries in $A$:

In [10]:
print(Type("list", natT))

nat list


All these can be combined in arbitrary ways. For example, the following is a type representing lists of functions that take a list of natural numbers as input, and returns a natural number:

In [11]:
print(Type("list", TFun(Type("list", natT), natT)))

(nat list => nat) list


A few methods are defined for working with function types:
- `is_fun()` returns whether the type is a function type.
- Given a type $A \To B$, `domain_type()` returns $A$ and `range_type()` returns $B$.
- Given a type $A_1 \To\cdots\To A_n\To B$, `strip_type()` returns the pair $[A_1,\dots,A_n], B$.

In [12]:
a = TFun(natT, boolT)
print(a.is_fun())
print(boolT.is_fun())
print(a.domain_type())
print(a.range_type())
print(a.strip_type())

b = TFun(natT, natT, boolT)
print(b.strip_type())

True
False
nat
bool
([Type(nat, [])], Type(bool, []))
([Type(nat, []), Type(nat, [])], Type(bool, []))


## Type variables

A *type variable* is a variable that can stand in for any type. We follow the convention of writing a type variable with name $a$ as `'a`. Type variables are constructed as follows:

In [13]:
print(TVar("a"))

'a


Type variables can be used as arguments to type constructors. For example, the following type represents all functions from type $a$ to type $b$:

In [14]:
print(TFun(TVar("a"), TVar("b")))

'a => 'b


Next, we introduce the important concepts of *substitution* and *matching*. A type with type variables can be considered as a *pattern* for producing types. If each type variable in the pattern is assigned a concrete value, the pattern can be *instantiated* to a concrete type (without type variables). We illustrate this with some examples.

In [15]:
p = TFun(TVar("a"), TVar("b"))
print(p)
print(p.subst({"a": natT, "b": boolT}))
print(p.subst({"a": TFun(natT, natT), "b": boolT}))
print(p.subst({"a": natT, "b": TFun(natT, boolT)}))

'a => 'b
nat => bool
(nat => nat) => bool
nat => nat => bool


In fact, we can assign a type variable to another type containing type variables. Note in this case, substitution of all variables is performed at the same time.

In [16]:
print(p.subst({"a": TVar("b"), "b": TVar("a")}))

'b => 'a


Matching can be considered as the dual of substitution. Given a pattern $p$ (a type containing type variables) and a type $t$ (with or without type variables), it determines whether $p$ can be instantiated to $t$, and returns the assignment of type variables in $p$ if it is possible.

In [17]:
print(p.match(TFun(natT, boolT)))

{'a': Type(nat, []), 'b': Type(bool, [])}


If it is impossible to instantiate $p$ to $t$, the match function throws a `TypeMatchException`:

In [18]:
p.match(natT)  # raises TypeMatchException

TypeMatchException: 

Note the same type variable can appear multiple times in a pattern. During matching, each occurrence of the type variable must be assigned to the same type.

In [19]:
q = TFun(Type("list", TVar("a")), TVar("a"))
print(q)
print(q.subst({"a": natT}))
print(q.match(TFun(Type("list", natT), natT)))

'a list => 'a
nat list => nat
{'a': Type(nat, [])}


Here is an example of a matching that failed because the two occurrences of `'a` correspond to different types ($nat$ and $bool$).

In [20]:
q.match(TFun(Type("list", natT), boolT))  # raises TypeMatchException

TypeMatchException: 

## Miscellaneous functions

`name` can be used to access the name of a type variable or constructor. `args` can be used to access the list of arguments (returned as a tuple):

In [21]:
a = TVar("a")
print(a.name)

b = Type("list", natT)
print(b.name)
print(b.args)

a
list
(Type(nat, []),)


`get_tvars()` returns the list of type variables in a type. `get_tsubs()` returns the list of all types appearing in a type.

In [22]:
a = TFun(TVar("a"), TVar("b"), natT)
print(a)
print(", ".join(str(t) for t in a.get_tvars()))
print(", ".join(str(t) for t in a.get_tsubs()))

'a => 'b => nat
'a, 'b
'a => 'b => nat, 'a, 'b => nat, 'b, nat
